In [1]:
import pprint
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:
c = ['sepal_length','sepal_width','petal_length','petal_width','target']
df = pd.read_csv('/kaggle/input/project-sc/iris.data',names=c)
df

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
i=0
for s in df['target'] :
  #print(s)
  if s=='Iris-setosa' :
    df['target'][i]=0
  elif s=='Iris-versicolor' :
    #print("1 2 3 4")
    df['target'][i]=1
  else :
   df['target'][i]=2
  i+=1
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
class FuzzyKNN(BaseEstimator, ClassifierMixin):
	def __init__(self, k=3, plot=False):
		self.k = k
		self.plot = plot


	def fit(self, X, y=None):
		self._check_params(X,y)
		self.X = X
		self.y = y

		self.xdim = len(self.X[0])
		self.n = len(y)

		classes = list(set(y))
		classes.sort()
		self.classes = classes

		self.df = pd.DataFrame(self.X)
		self.df['y'] = self.y

		self.memberships = self._compute_memberships()

		self.df['membership'] = self.memberships

		self.fitted_ = True
		return self


	def predict(self, X):
		if self.fitted_ == None:
			raise Exception('predict() called before fit()')
		else:
			m = 2
			y_pred = []

			for x in X:
				neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)

				votes = {}
				for c in self.classes:
					den = 0
					for n in range(self.k):
						dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
						den += 1 / (dist ** (2 / (m-1)))

					neighbors_votes = []
					for n in range(self.k):
						dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
						num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))

						vote = num/den
						neighbors_votes.append(vote)
					votes[c] = np.sum(neighbors_votes)

				pred = max(votes.items(), key=operator.itemgetter(1))[0]
				y_pred.append((pred, votes))

			return y_pred


	def score(self, X, y):
		if self.fitted_ == None:
			raise Exception('score() called before fit()')
		else:
			predictions = self.predict(X)
			y_pred = [t[0] for t in predictions]
			confidences = [t[1] for t in predictions]

			return accuracy_score(y_pred=y_pred, y_true=y)


	def _find_k_nearest_neighbors(self, df, x):
		X = df.iloc[:,0:self.xdim].values

		df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]

		df.sort_values(by='distances', ascending=True, inplace=True)
		neighbors = df.iloc[0:self.k]

		return neighbors


	def _get_counts(self, neighbors):
		groups = neighbors.groupby('y')
		counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}

		return counts


	def _compute_memberships(self):
		memberships = []
		for i in range(self.n):
			x = self.X[i]
			y = self.y[i]

			neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
			counts = self._get_counts(neighbors)

			membership = dict()
			for c in self.classes:
				try:
					uci = 0.49 * (counts[c] / self.k)
					if c == y:
						uci += 0.51
					membership[c] = uci
				except:
					membership[c] = 0

			memberships.append(membership)
		return memberships


	def _check_params(self, X, y):
		if type(self.k) != int:
			raise Exception('"k" should have type int')
		if self.k >= len(y):
			raise Exception('"k" should be less than no of feature sets')
		if self.k % 2 == 0:
			raise Exception('"k" should be odd')

		if type(self.plot) != bool:
			raise Exception('"plot" should have type bool')

In [24]:
X=df[['sepal_length','sepal_width','petal_length','petal_width']]
y=df['target'].tolist()
x1=X['sepal_length'].tolist()
x2=X['sepal_width'].tolist()
x3=X['petal_length'].tolist()
x4=X['petal_width'].tolist()
X=np.array(list(zip(x1,x2,x3,x4)))
xTrain, xTest, yTrain, yTest = train_test_split(X,y,test_size=0.7,random_state=74)

custModel = FuzzyKNN()
custModel.fit(xTrain, yTrain)
c=cross_val_score(cv=20, estimator=custModel, X=xTest, y=yTest)
print("cross validation accuarcies for 20 iterations :",c)
print("avg accuarcy :",c.mean())

cross validation accuarcies for 20 iterations : [1.         0.83333333 1.         1.         1.         1.
 1.         1.         1.         0.8        1.         1.
 1.         0.8        1.         1.         1.         1.
 1.         1.        ]
avg accuarcy : 0.9716666666666667


In [6]:
c = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34']
df2 = pd.read_csv('/kaggle/input/project-sc/ionosphere.data',names=c)
df2

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


In [7]:
i=0
for s in df2['34'] :
  #print(s)
  if s=='g' :
    #print("lknlnlkn")
    df2['34'][i]=0
  else :
   df2['34'][i]=1
  i+=1
df2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,0
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,1
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,0
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,1
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,0
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,0
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,0
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,0


In [8]:
X2=df2[c]
y2 = df2['34'].tolist()
x1=X2['0'].tolist()
x2=X2['1'].tolist()
x3=X2['2'].tolist()
x4=X2['3'].tolist()
x5=X2['4'].tolist()
x6=X2['5'].tolist()
x7=X2['6'].tolist()
x8=X2['7'].tolist()
x9=X2['8'].tolist()
x10=X2['9'].tolist()
x11=X2['10'].tolist()
x12=X2['11'].tolist()
x13=X2['12'].tolist()
x14=X2['13'].tolist()
x15=X2['14'].tolist()
x16=X2['15'].tolist()
x17=X2['16'].tolist()
x18=X2['17'].tolist()
x19=X2['18'].tolist()
x20=X2['19'].tolist()
x21=X2['20'].tolist()
x22=X2['21'].tolist()
x23=X2['22'].tolist()
x24=X2['23'].tolist()
x25=X2['24'].tolist()
x26=X2['25'].tolist()
x27=X2['26'].tolist()
x28=X2['27'].tolist()
x29=X2['28'].tolist()
x30=X2['29'].tolist()
x31=X2['30'].tolist()
x32=X2['31'].tolist()
x33=X2['32'].tolist()
x34=X2['33'].tolist()
x35=X2['34'].tolist()

X2=np.array(list(zip(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35)))

xTrain, xTest, yTrain, yTest = train_test_split(X2,y2,test_size=0.7,random_state=500)
custModel = FuzzyKNN()
custModel.fit(xTrain, yTrain)
c=cross_val_score(cv=20, estimator=custModel, X=xTest, y=yTest)
print(c)
print(c.mean())

[1.         0.76923077 0.92307692 0.92307692 0.76923077 0.84615385
 0.91666667 0.91666667 0.91666667 0.91666667 0.75       1.
 1.         0.91666667 0.91666667 0.83333333 0.75       1.
 1.         0.75      ]
0.8907051282051281
